# morvan_pytorch GAN
https://morvanzhou.github.io/tutorials/machine-learning/torch/4-06-GAN/
训练一个GAN画二次曲线，生成的过程如下图：

![莫凡gif](https://morvanzhou.github.io/static/results/torch/4-6-1.gif)




In [19]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
"""
两次bakward()之前必须重新计算，否则梯度计算会叠加
"""
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import sys

torch.manual_seed(1)
np.random.seed(1)
torch.autograd.set_detect_anomaly(True)

# Hyper Parameters
BATCH_SIZE = 64
LR_G = 0.001  # learning rate for generator
LR_D = 0.001  # learning rate for discriminator
N_IDEAS = 5  # think of it as number of ideas for generating an art work
ART_COMPONENTS = 15  # it could be the total point G can draw in the canvas
PAINT_POINTS = np.vstack([np.linspace(-1, 1, ART_COMPONENTS) for _ in range(BATCH_SIZE)])


def artist_works():
    a = np.random.uniform(1, 2, size=BATCH_SIZE).reshape((-1, 1))
    paintins = a * np.power(PAINT_POINTS, 2) + (a - 1)
    paintins = torch.tensor(paintins, dtype=torch.float)
    return paintins

G = nn.Sequential(
    nn.Linear(N_IDEAS, 128),
    nn.ReLU(),
    nn.Linear(128, ART_COMPONENTS),
)

D = nn.Sequential(
    nn.Linear(ART_COMPONENTS, 128),
    nn.ReLU(),
    nn.Linear(128, 1),
    nn.Sigmoid(),  # the prbability that the art work is made by artist
)

optim_G = torch.optim.Adam(G.parameters(), lr=LR_G)
optim_D = torch.optim.Adam(D.parameters(), lr=LR_D)

plt.ion()  # it is about continuous plotting

for step in range(int(1e4)):
    artist_paintings = artist_works()
    G_ideas = torch.randn(BATCH_SIZE, N_IDEAS, requires_grad=True)
    G_paintings = G(G_ideas)
    
    prob_artist0 = D(artist_paintings)  # D tries to increase the probability
    prob_artist1 = D(G_paintings.detach())  # D tries to decrease the probability
    D_loss = - torch.mean(torch.log(prob_artist0) + torch.log(1-prob_artist1))
    optim_D.zero_grad()
    D_loss.backward()
    optim_D.step()
    
    prob_artist1 = D(G_paintings)  # D tries to decrease the probability
    G_loss = torch.mean(torch.log(1-prob_artist1))
    optim_G.zero_grad()
    G_loss.backward()
    optim_G.step()
    
    if step % 40 == 0:  # plotting
        plt.cla()
        plt.plot(PAINT_POINTS[0], G_paintings.data.numpy()[0], c='#4AD631', lw=3, label='Generated painting',)
        plt.plot(PAINT_POINTS[0], 2 * np.power(PAINT_POINTS[0], 2) + 1, c='#74BCFF', lw=3, label='upper bound')
        plt.plot(PAINT_POINTS[0], 1 * np.power(PAINT_POINTS[0], 2) + 0, c='#FF9359', lw=3, label='lower bound')
        plt.text(-.5, 2.3, 'D accuracy=%.2f (0.5 for D to converge)' % prob_artist0.data.numpy().mean(), fontdict={'size': 13})
        plt.text(-.5, 2, 'D score= %.2f (-1.38 for G to converge)' % -D_loss.data.numpy(), fontdict={'size': 13})
        plt.ylim((0, 3));plt.legend(loc='upper right', fontsize=10);plt.draw();plt.pause(0.01)
        

plt.ioff()
plt.show()

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [14]:
b = np.random.uniform(1, 2, size=BATCH_SIZE).

# 测试

## 两次backward()必须要重新计算，否则计算的是二阶导数

In [25]:
import torch
import torch.nn as nn

class Function(torch.nn.Module):
    def __init__(self):
        super(Function, self).__init__()
    def forward(self, input):
        output = 2*input[0]**2 + input[1]
        return output

func = Function()

x= torch.tensor([2.0, 1.0], requires_grad=True)
y = func(x)
y.backward(retain_graph=True)

print('After first bakward: ', x.grad)

y.backward(retain_graph=True)
print('After second bakward: ', x.grad)

x.grad.zero_()
y.backward(retain_graph=True)
print('After third bakward but set the gradient zero: ', x.grad)

After first bakward:  tensor([8., 1.])
After second bakward:  tensor([16.,  2.])
After third bakward but set the gradient zero:  tensor([8., 1.])
